In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

df = pd.read_csv('data.csv')

# filter dataframe
df = df.loc[(df['Vuosi']<2022)]
df = df.dropna()

data = []
# loop through the rows using iterrows()
for index, row in df.iterrows():
    val = np.array([row['val']])
    kk = np.array([row['Kk']])
    lampo = np.array([row['Kuukauden keskilämpötila (degC)']])
    i = np.concatenate((kk,lampo, val), axis=0)
    data.append(i)

data = np.array(data)

np.random.shuffle(data)

# Calculate the number of rows to include in the first array (80%)
percent_to_include = 0.8
num_rows_training = int(percent_to_include * data.shape[0])

# Split the original array into two arrays
training = data[:num_rows_training, :]
test = data[num_rows_training:, :]

# Split training and test data to X_train, y_train and X_test and y_test
# Extract the last value from every column
y_train = training[:, -1]
X_train = training[:, :-1]

y_test = test[:, -1]
X_test = test[:, :-1]

model = LinearRegression()

model.fit(X_train, y_train)

# Calculate the average influenzas per month from the input data
dfave = df[['Kk','val']]
grouped = dfave.groupby('Kk')
average_influenzas = grouped.mean()
averagemonth = average_influenzas.to_numpy()

# Calculate the average influenzas per year from the input data
dfaveyear = df[['val']]
averageyear = dfaveyear.mean()


#
# Risk evaluation
#
# Risk formula = mean of the average influenzas per month + predicted influenzas
# Risk formula is compared to avarage influenzas per year
#
# There are three risk classes which are determined as follows:
# "Low risk" Risk formula is less than the average influenzas per year - 100 influenzas
# "Medium risk" Risk formula is between  the average influenzas per year - 100 influenzas and 
#                                        the average influenzas per year + 100 influenzas
# "High risk" Risk formula is greater than the average influenzas per year + 100 influenzas
#


month = int(input("Please type month (type -1 to exit): "))
while month != -1:
    if month == -1:
        month = 0
        break
    while month <  1 or month > 12:
        print('Month is between 1 and 12')
        month = int(input("Please type month: "))
    temp = int(input("Please type temperature: "))
    pre =model.predict(np.array([[month, temp]]))
    ave =averagemonth[month-1]
    #
    # risk formula = (the average influenzas per month + predicted influenzas) / 2
    #
    risk = (ave + pre)/2
    if (risk < averageyear -100).any():
        print("Low risk")
    elif (risk >= averageyear - 100).any and (risk < averageyear + 100).any():
        print("Medium risk")
    else:
        print("High risk")
        
    month = int(input("Please type month: "))
 
    

Please type month (type -1 to exit): 1
Please type temperature: -3
High risk
Please type month: 4
Please type temperature: 4
Medium risk
Please type month: 5
Please type temperature: 5
Medium risk
Please type month: 9
Please type temperature: -3
Medium risk
Please type month: -1
